# Project 101
### PARTE 2.3 - Modelos Redes Neuronales

### Contenido
- [Modelado](#modelos)

    - [**2.3.1 Glounts Deep Ar**](#1)
        - [Definicion del modelo](#def)
        - [Modelado con variable exogena](#exo)
        - [Comparacion de Resultados](#1.3)

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import funciones_utiles
from funciones_utiles import DataPrep, Metrics, guardar_metricas

import time
time.clock = time.time

from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.common import ListDataset
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.torch.distributions import NegativeBinomialOutput
from gluonts.evaluation import make_evaluation_predictions

from sklearn.metrics import mean_squared_error, mean_absolute_error

from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
import plotly.express as px
import dash_bootstrap_components as dbc


__Importamos los df__

In [22]:
comparison_tbl= pd.read_csv('Data/Modelado/comparison_tbl.csv')

In [23]:
df= DataPrep().unificar_df()
df['y']=df['y'].astype(int)
df['ds']= pd.to_datetime(df['ds'])
df.head()

,ds,unique_id,y
0,2019-01-06,impo_1,1950
1,2019-01-13,impo_1,2402
2,2019-01-20,impo_1,2782
3,2019-01-27,impo_1,3263
4,2019-02-03,impo_1,3843


In [24]:
df.columns

Index(['ds', 'unique_id', 'y'], dtype='object')

#### 2.3.1 Glounts Deep Ar


__Definimos el set de train y test__

In [25]:
train = df.loc[df['ds'] < '2022-11-01']
valid = df.loc[df['ds'] >= '2022-11-01']

In [26]:
train_ds = PandasDataset.from_long_dataframe(train,
                                             target='y',
                                             item_id='unique_id', 
                                             timestamp='ds',
                                             freq='W')

__Definicion del modelo DeepAr__

La distribucion por default de DeepAR es un distribucion t-student, pero para modelar los datos es preferible que sea una Negative Binomial ya que permite obtener valores pertenecientes a los Reales Positivos, y no predicciones negativas como ocurre con la distribucion por deafault. Esta informacion es conocidad gracias al domain knowledge que no es posible que hayan valores de kilos negativos sino que pueden tomar valores 0 o positivos. 

In [27]:
tic = time.clock()
estimator = DeepAREstimator(freq='W', #la frecuencia de los datos es semanal por lo que el modelo debe ser semanal tambien
                            context_length=10, # el modelo va a usar las ultimas 10 semanas para predecir las siguientes 10, window de 10
                            prediction_length=10, # va a predecir las proximas 10 semanas 
                            num_layers=4, #el modelo posee tres capas con un default de 40 nodos por capa 
                            dropout_rate= 0.2, #seteo de 20% de las units en una layer en cero de forma random
                            trainer_kwargs={ 'max_epochs':10},#210 epochs
                            distr_output= NegativeBinomialOutput()) #el default es Distribucion t-student, pero al usar negative binomial permite que las predicciones no tomen valores negativos
                        
predictor = estimator.train(train_ds, num_workers= 2)

toc =  time.clock() #frenamos el cronometro y a continuacion creamos la variable que guarda el tiempo transcurrido desde que empezo a correr el modelo hasta q termino
exetime = '{0:.4f}'.format(toc-tic)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name  | Type        | Params | In sizes | Out sizes   
----------------------------------------------------------------
0 | model | DeepARModel | 50.0 K | ?        | [1, 100, 10]
----------------------------------------------------------------
50.0 K    Trainable params
0         Non-trainable params
50.0 K    Total params
0.200     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 20.82733 (best 20.82733), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_174\\checkpoints\\epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 12.25746 (best 12.25746), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_174\\checkpoints\\epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' was not in top 1
Epoch 3, global step 200: 'train_loss' was not in top 1
Epoch 4, global step 250: 'train_loss' was not in top 1
Epoch 5, global step 300: 'train_loss' was not in top 1
Epoch 6, global step 350: 'train_loss' was not in top 1
Epoch 7, global step 400: 'train_loss' was not in top 1
Epoch 8, global step 450: 'train_loss' was not in top 1
Epoch 9, global step 500: 'train_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=10` reached.


In [28]:
pred = list(predictor.predict(train_ds))

all_preds = list()
for item in pred:
    unique_id = item.item_id
    p = item.samples.mean(axis=0)
    p10 = np.percentile(item.samples, 10, axis=0)
    p90 = np.percentile(item.samples, 90, axis=0)
    p25 = np.percentile(item.samples, 25, axis=0)
    p75 = np.percentile(item.samples, 75, axis=0)
    dates = pd.date_range(start=item.start_date.to_timestamp(), periods=len(p), freq='W')
    family_pred = pd.DataFrame({'ds': dates, 
                                'unique_id': unique_id,
                                'pred': p,
                                'p25': p25,
                                'p75': p75,
                                'p10': p10, 
                                'p90': p90})
    all_preds += [family_pred]
all_preds = pd.concat(all_preds, ignore_index=True)

all_preds = all_preds.merge(valid, on=['ds', 'unique_id'], how='left')

all_preds.dropna(inplace=True)

Metrics(all_preds['y'], all_preds['pred']).calculate_smape()

0.7921637231899379

In [31]:
all_preds

,ds,unique_id,pred,p25,p75,p10,p90,y
0,2022-11-06,expo_1,1906.540039,445.25,2785.00,160.7,3984.2,2720.0
1,2022-11-13,expo_1,1914.739990,420.25,2872.75,133.5,4663.5,290.0
2,2022-11-20,expo_1,1741.180054,362.50,2422.50,106.8,3980.9,906.0
3,2022-11-27,expo_1,1564.349976,397.25,2251.75,130.8,3542.0,2408.0
4,2022-12-04,expo_1,1753.459961,356.75,2635.50,76.6,3607.9,326.0
...,...,...,...,...,...,...,...,...
114,2022-12-04,impo_7,451.109985,118.00,631.75,49.7,1127.9,278.0
115,2022-12-11,impo_7,545.140015,149.50,678.75,39.0,1299.1,182.0
116,2022-12-18,impo_7,594.190002,149.50,842.25,39.7,1559.7,260.0
117,2022-12-25,impo_7,508.119995,109.00,761.75,42.5,1262.5,409.0


__Resultados Obtenidos: Deep AR__

In [12]:
app = Dash(external_stylesheets=[dbc.themes.SLATE])



dropdown = dcc.Dropdown(
    id="ticker",
    options=[{"label": unique_id, "value": unique_id} for unique_id in df["unique_id"].unique()],
    value="impo_1",
    clearable=False, 
    style={'background-color': 'grey'}
)

app.layout = html.Div([
    html.H4('Deep AR Resultados'),
    html.P("Seleccionar motivo:"),
    dropdown,
    dcc.Graph(id="time-series-chart"),
])


@app.callback(
    Output("time-series-chart", "figure"), 
    Input("ticker", "value"))

def display_time_series(ticker):
    df_pivot = df.pivot(index='ds', columns='unique_id', values='y')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    
    fig = px.line(df_pivot, x='ds', y=ticker, title=ticker, template='plotly_dark')
    fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)',
                      paper_bgcolor='rgba(0, 0, 0, 0)')
    
    #bandas de percentiles 
    p_ = all_preds.loc[all_preds['unique_id'] == ticker]
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p10'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p10'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p90'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p90', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p25'], mode='lines', line=dict(color='orange'), name='p25'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p75'], mode='lines', line=dict(color='orange'), name='p75', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['pred'], mode='lines', line=dict(color='orange'), name='Forecast'))
    return fig


if __name__ == "__main__":
    app.run_server(debug=True, port=8080, mode='inline')

__Guardamos las métricas para cada una de las series__

In [13]:
comparison_tbl=guardar_metricas(comparison_tbl, all_preds, exetime, 'Glounts Deep AR')

NameError: name 'comparison_tbl' is not defined

In [9]:
best_models = comparison_tbl.groupby('Serie').apply(lambda group: group[group['MAE'] == group['MAE'].min()])
best_models.drop(columns='Serie', inplace=True)
best_models = best_models.groupby('Serie').first().reset_index()
best_models.tail(12)

,Serie,Modelo,MAE,RMSE,sMAPE,Processing Time
18,impo_3,Glounts Deep AR,27.063334,32.217003,1.075632,50.5338
2,Impo 3,"SARIMAX ([1, 2], 0, [5])",42.912352,77.237038,1.431710,2.6416
14,expo_4,Glounts Deep AR,44.653334,66.411531,0.559923,50.5338
15,expo_5,Glounts Deep AR,51.861113,58.219287,0.282507,50.5338
19,impo_4,Glounts Deep AR,65.242221,72.757716,0.789762,50.5338
10,Expo 4,"SARIMAX ([1, 2], 1, [2])",96.732396,137.010334,0.726832,1.6148
6,Impo 7,"SARIMAX ([1, 2], 0, [10])",141.687432,183.721246,0.342107,8.5445
3,Impo 4,"SARIMAX ([2], 0, [1])",185.779630,213.952750,0.812257,0.9748
9,Expo 3,"SARIMAX ([3], 0, [1])",199.795894,367.519378,0.367942,3.9468
22,impo_7,Glounts Deep AR,221.135552,239.400618,0.558031,50.5338


#### 2.3.1 Glounts Deep AR - Variable exógena

Cotizacion dolar oficial

Obtenida de: https://es.investing.com/currencies/usd-ars-historical-data

In [14]:
df_cot_dolar_blue= pd.read_csv('Data/Datos históricos USD_ARSB.csv')
df_cot_dolar_blue['Fecha'] = pd.to_datetime(df_cot_dolar_blue['Fecha'], format='%d.%m.%Y')
df_cot_dolar_blue= df_cot_dolar_blue[['Fecha','Último']]
df_cot_dolar_blue['Último']= df_cot_dolar_blue['Último'].str.replace(',', '.')
df_cot_dolar_blue['Último']= df_cot_dolar_blue['Último'].astype(float)

In [15]:
df_cot_dolar_blue.tail()

,Fecha,Último
208,2019-02-03,37.38
209,2019-01-27,37.62
210,2019-01-20,38.12
211,2019-01-13,38.88
212,2019-01-06,39.00


In [16]:
df_merged = df.merge(df_cot_dolar_blue, left_on='ds', right_on='Fecha', how='left')
df_merged = df_merged.rename(columns={'Último': 'cotizacion'}).drop('Fecha', axis=1)
df_merged['cotizacion']=df_merged['cotizacion'].astype(int)

In [17]:
df_merged[df_merged['ds']=='2023-01-01']

,ds,unique_id,y,cotizacion
208,2023-01-01,impo_1,2813,352
417,2023-01-01,impo_2,237,352
626,2023-01-01,impo_3,50,352
835,2023-01-01,impo_4,50,352
1044,2023-01-01,impo_5,1111,352
1080,2023-01-01,impo_6,890,352
1289,2023-01-01,impo_7,201,352
1498,2023-01-01,expo_1,1617,352
1706,2023-01-01,expo_2,281,352
2120,2023-01-01,expo_4,41,352


__Definimos el set de train y test__

In [30]:
train= df_merged
valid = df_merged[df_merged['ds'] >= '2022-11-01']

#cuando agregamos una variable exogena la misma debe ser input en la fase de test, es decir debe ser una variable conocida a futuro.
#Aunque en la practica es casi inviable conocer la cotizacion del dolar a futuro se podria crear un segundo modelo que prediga la misma, 
#pero en esta instancia a modo de prueba correre el modelo bajo el supuesto que podremos agregar la cotizacion del dolar futuro en el modelo 
#una vez que este en produccion

In [23]:
train_ds = PandasDataset.from_long_dataframe(train,
                                             target='y', 
                                             item_id='unique_id', 
                                             timestamp='ds',
                                             freq='W', 
                                             feat_dynamic_real=['cotizacion']) #agregamos la variable exogena 
#future_lenght = 10 
#pasarle al train todo el dataset, la variable target hasta el 1 de noviembre, y necesitos 10 valores de cot hacia el futuro 
#como es autoregresivo necesita los valores futuros, valor del dolar hasta hoy 
#prediga las dos variables -> deep VAR -> predecir las dos cosas, target + exogenas => estimar por fuera el dolar -> input nuevo modelo 
#hacer la prueba si realmente mejora evaluar las opciones => el valor observado futuro nunca lo voy a tener -> valor estimado != valor futuro 
#variable categorica estatica => pais de origen del motivo -> USA, EUROPA, CHINA etc -> static categorical no varia en el tiempo

__Definicion del Modelo DeepAr - feat dynamic real__

In [24]:
tic = time.clock()
estimator2 = DeepAREstimator(freq='W', #la frecuencia de los datos es semanal por lo que el modelo debe ser semanal tambien
                            context_length=9, # el modelo va a usar las ultimas 9 semanas para predecir la siguientes 10
                            prediction_length=9, # va a predecir las proximas 9 semanas 
                            num_layers=4, #el modelo posee 4 capas con un default de 40 nodos por capa 
                            dropout_rate= 0.2, #seteo de 20% de las units en una layer en cero de forma random, el default es 0.1
                            trainer_kwargs={ 'max_epochs':10},#120, #en vez de poner un early stopping defino un max epoch de 30 ((210)
                            num_feat_dynamic_real= int(1), 
                            distr_output= NegativeBinomialOutput())  #el default es Distribucion t-student, pero al usar negative binomial permite que las predicciones no tomen valores negativos

predictor2 = estimator2.train(train_ds, num_workers=2)

toc =  time.clock() #frenamos el cronometro y a continuacion creamos la variable que guarda el tiempo transcurrido desde que empezo a correr el modelo hasta q termino
exetime = '{0:.4f}'.format(toc-tic)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning:

You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.


  | Name  | Type        | Params | In sizes | Out sizes  
---------------------------------------------------------------
0 | model | DeepARModel | 50.2 K | ?        | [1, 100, 9]
---------------------------------------------------------------
50.2 K    Trainable params
0         Non-trainable params
50.2 K    Total params
0.201     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 12.67451 (best 12.67451), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_131\\checkpoints\\epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' was not in top 1
Epoch 2, global step 150: 'train_loss' was not in top 1
Epoch 3, global step 200: 'train_loss' reached 12.49430 (best 12.49430), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_131\\checkpoints\\epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' was not in top 1
Epoch 5, global step 300: 'train_loss' reached 9.76146 (best 9.76146), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\lightning_logs\\version_131\\checkpoints\\epoch=5-step=300.ckpt' as top 1
Epoch 6, global step 350: 'train_loss' was not in top 1
Epoch 7, global step 400: 'train_loss' reached 9.68130 (best 9.681

In [25]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_ds,  
    predictor=predictor2, 
)

tss = list(ts_it)
forecasts = list(forecast_it)

In [31]:
all_preds2 = list()
for item in forecasts:
    unique_id = item.item_id
    p = item.samples.mean(axis=0)
    p10 = np.percentile(item.samples, 10, axis=0)
    p90 = np.percentile(item.samples, 90, axis=0)
    p25 = np.percentile(item.samples, 25, axis=0)
    p75 = np.percentile(item.samples, 75, axis=0)
    dates = pd.date_range(start=item.start_date.to_timestamp(), periods=len(p), freq='W')
    family_pred = pd.DataFrame({'ds': dates, 
                                'unique_id': unique_id,
                                'pred': p,
                                'p25': p25,
                                'p75': p75,
                                'p10': p10, 
                                'p90': p90})
    all_preds2 += [family_pred]
all_preds2 = pd.concat(all_preds2, ignore_index=True)
valid.drop(columns='ds', inplace=True)
all_preds2 = all_preds2.merge(valid, on=['ds', 'unique_id'], how='left')

all_preds2.dropna(inplace=True)

Metrics(all_preds2['y'], all_preds2['pred']).wmape()

C:\Users\Bravo15\AppData\Local\Temp\ipykernel_18072\197479032.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.7393604936422814

__Resultados Obtenidos: Deep AR - feat dynamic real__

In [32]:
app2 = Dash(external_stylesheets=[dbc.themes.SLATE])


dropdown = dcc.Dropdown(
    id="ticker",
    options=[{"label": unique_id, "value": unique_id} for unique_id in df["unique_id"].unique()],
    value="impo_1",
    clearable=False, 
    style={'background-color': 'grey'}
)

app2.layout = html.Div([
    html.H4('Deep AR - Feat dynamic real Resultados'),
    html.P("Seleccionar motivo:"),
    dropdown,
    dcc.Graph(id="time-series-chart"),
])


@app2.callback(
    Output("time-series-chart", "figure"), 
    Input("ticker", "value"))

def display_time_series(ticker):
    df_pivot = df.pivot(index='ds', columns='unique_id', values='y')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    
    fig = px.line(df_pivot, x='ds', y=ticker, title=ticker, template='plotly_dark')
    fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)',
                      paper_bgcolor='rgba(0, 0, 0, 0)')
    
    #bandas de percentiles 
    p_ = all_preds2.loc[all_preds2['unique_id'] == ticker]
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p10'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p10'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p90'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p90', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p25'], mode='lines', line=dict(color='orange'), name='p25'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p75'], mode='lines', line=dict(color='orange'), name='p75', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['pred'], mode='lines', line=dict(color='orange'), name='Forecast'))
    return fig


if __name__ == "__main__":
    app2.run_server(debug=True, port=8081, mode='inline')

__Guardamos las métricas para cada una de las series__

In [33]:
comparison_tbl=guardar_metricas(comparison_tbl, all_preds, exetime, 'Glounts Deep AR feat dynamic variable')

NameError: name 'comparison_tbl' is not defined

In [155]:
best_models = comparison_tbl.groupby('Serie').apply(lambda group: group[group['RMSE'] == group['RMSE'].min()])
best_models.drop(columns='Serie', inplace=True)
best_models = best_models.groupby('Serie').first().reset_index()
best_models.tail(12)

,Serie,Modelo,MAE,RMSE,sMAPE,Processing Time
11,expo_1,Glounts Deep AR,736.127767,826.523713,0.649992,831.7742
12,expo_2,Glounts Deep AR,782.855559,1038.220522,1.518301,831.7742
13,expo_3,Glounts Deep AR,203.240002,215.670565,0.559893,831.7742
14,expo_4,Glounts Deep AR,51.296667,71.753629,0.655892,831.7742
15,expo_5,Glounts Deep AR,37.885553,52.572408,0.213746,831.7742
16,impo_1,Glounts Deep AR,926.861084,1054.689090,0.302871,831.7742
17,impo_2,Glounts Deep AR,104.382222,110.197790,0.261960,831.7742
18,impo_3,Glounts Deep AR,25.522222,29.375341,0.965812,831.7742
19,impo_4,Glounts Deep AR,20.353333,25.710064,0.469926,831.7742
20,impo_5,Glounts Deep AR,533.032227,799.136363,0.512526,831.7742


**Conclusion:**  A pesar de agregar la variable exogena del tipo de cambio, aunque los resultados del modelo no mejoran y aun el modelo Glounts Deep AR permite obtener mejores predicciones. Comparando los resultados entre el primer modelo Deep Ar y el segundo Deep Ar con feat dynamic real, se observa que ambos no logran captar las tendencias y cambios drasticos existentes en los datos. Si bien los resultados son los mejores obtenidos hasta el momento comparando con los modelos clasicos y de ML, aun asi el modelo permite obtener predicciones con cierto margen de error pero aun no logra captar los cambios abruptos que poseen las series. En gran medida se debe a la naturaleza del modelo Deep AR, ya que como su nombre indica al ser un modelo autoregresivo busca patrones en valores historicos de la serie buscando hacer inferencias con distribuciones conocidas. Pero aun asi, como los datos presentes son erraticos y no revelan una distribucion aparente este tipo de modelo no parece ser suficiente. A continuacion, presentare otro tipo de modelo de redes neuronales el cual se basa en en redes neuronales recurrentes y long short term memory, de esta forma logra captar estos "changing points" o cambios erraticos que presentan este tipo de series. 